In [27]:
import random
from IPython.display import clear_output

#Constants
BET_RATIO = 1.5 # Player will win bet money * BET_RATIO

class Card():   
    def __init__(self, category, name, value):
        # private attributes, Python uses the __ as convention
        self.__category = category 
        self.__name = name
        self.__value = value
        
    def __str__(self):
        return f"{self.__name} of {self.__category} ({self.__value})"
    
    def set_value(self, value):
        self.__value = value
        
    def get_value(self):
        return self.__value
    
    def get_name(self):
        return self.__name
    
    def get_category(self):
        return self.__category


class Deck():
    CARD_CATEGORIES = ["Spades", "Hearts", "Diamonds", "Clubs"]
    CARD_VALUES = {"Ace":[1,11], "Two":2, "Three":3, "Four":4, "Five":5, "Six":6, "Seven":7, "Eight":8, "Nine":9, "Ten":10, "Jack":10, "Queen":10, "King":10}
    
    def __init__(self):
        self.total_cards = 52
        self.cards = []
        
        # Create our deck for blackjack and shuffle it
        for card in self.CARD_VALUES:
            for category in self.CARD_CATEGORIES:
                self.cards.append(Card(category, card, self.CARD_VALUES[card]))
        
        # Shuffle our deck
        self.shuffle()
        
    def shuffle(self):
        random.shuffle(self.cards)
    
    def top_card(self):
        # Takes the first card of the deck and returns it
        # No need to control the case of the deck being empty, this will never happen in this game
        self.total_cards-=1
        return self.cards.pop(0)
    
    def __str__(self):
        # Pretty sure this is not the way to do this LUL
        print(f"Total cards:{self.total_cards}")
        for card in self.cards:
            print(card)
        return ""
    
class Hand():
    def __init__(self):
        self.__cards = [] # private attribute
    
    def draw(self, deck):
        self.__cards.append(deck.top_card())
    
    def total_value(self):
        return sum(card.get_value() for card in self.__cards)
    
    def get_cards(self):
        return self.__cards


# TODO: this should be an abstract class
class Player():
    def __init__(self, savings):
        self.savings = savings
        self._hand = Hand()
        pass
      
    def draw(self,deck):
        self._hand.draw(deck)
    
    def print_hand(self, hide_second_card=False):
        # Prints the hand of the player
        if hide_second_card:
            for index,card in enumerate(self._hand.get_cards()):
                if index==1:
                    print("X|", end = '')
                else:
                    print(f"{card}|", end = '')
        else:
            for card in self._hand.get_cards():
                print(f"{card}|", end = '') 
    
    def define_aces_values(self):
        # Implement this method, the player has to decide if he will use their aces
        # as 1 or 11
        for card in self._hand.get_cards():
            condition = card.get_name() == "Ace" and card.get_value() == [1,11]            
            if(condition):
                # We need to ask the player to define the value for this card and update it
                ace_value = Utils.get_ace_value(card)
                card.set_value(ace_value)
    
    def busted(self):
        return self._hand.total_value() >= 21
    
    
class HumanPlayer(Player):
    def __init__(self,savings=100):
        Player.__init__(self,savings)
        self.bet_money = 0
    
    def stand():
        pass
    
    def can_bet(self,amount):
        return amount <= self.savings
    
    def bet(self,amount):
        # Update bet money here
        self.savings -= amount
        self.bet_money = amount
    
class Dealer(Player):
    def __init__(self, savings=5000):
        Player.__init__(self,savings)
    
class Utils:
    def capture_bet():
        while True:
            try:
                bet = int(input("Place your bet: "))
                return bet
            except:
                continue
    
    '''
    Return 0 if the player decides to stand or 1 if he wants to hit 
    '''
    def wants_to_hit():
        while True:
            hit_or_stand = input("Would you like to hit[h] or stand[s]?")
            if(hit_or_stand.lower() in ["hit", "h"]):
                return 1
            elif(hit_or_stand.lower() in ["stand", "s"]):
                return 0
            else:
                print("Invalid input, only 'h'/'hit' or 's'/'stand' are valid")
                continue

    def print_state(deck, dealer, player1, hide_second_card=False):
        clear_output()
        # Prints out the board with the state
        print("")
        print("Dealer")
        print("--------")
        print(f"Cards in deck: {deck.total_cards}")
        print(f"Savings: ${dealer.savings}")
        dealer.print_hand(hide_second_card)

        print("")
        print("")

        print("Player1")
        print("--------")
        print(f"Savings: ${player1.savings}")  
        print(f"Bet: ${player1.bet_money}")
        player1.print_hand()
        print("")
    
    def get_ace_value(card):
        while True:
            try:
                ace_value = int(input(f"Will you use {card.get_name()} of {card.get_category()} as 1 or 11?:"))

                if ace_value not in [1,11]:
                    print("Invalid value. It should be 1 or 11!")
                    pass
                    continue
                else:
                    return ace_value
            except:
                continue
        

# MAIN FUNCTION
def testingFunction():
    my_deck = Deck()
    player1 = HumanPlayer(500)
    dealer = Dealer()
    
    player1.draw(my_deck)
    player1.draw(my_deck)   
    dealer.draw(my_deck)
    dealer.draw(my_deck)
    
    
    
    player1.define_aces_values()
    Utils.print_state(my_deck, dealer, player1, True)



def play():
    print("---------- WELCOME TO BLACKJACK 1.0 ----------")
    
    # This method will include the main loop for the game
    # while True:
    # Initialize main objects
    my_deck = Deck()
    player1 = HumanPlayer(500)
    dealer = Dealer()
    
    # Ask Player1 for their bet
    player1_bet = Utils.capture_bet()
    while not player1.can_bet(player1_bet):
        print(f"You only have ${player1.savings}. Go for a lower bet!")
        player1_bet = Utils.capture_bet()
    player1.bet(player1_bet)
    
    # Deal two cards to the Dealer and two cards to the Player
    player1.draw(my_deck)
    player1.draw(my_deck)   
    dealer.draw(my_deck)
    dealer.draw(my_deck)
    
    # Print current state of the board
    Utils.print_state(my_deck, dealer, player1, True)

    # Analyze if there're any aces already drawn by him, ask him what value will be use
    player1.define_aces_values()
    
    # Reprint board in case anything changed
    Utils.print_state(my_deck, dealer, player1, True)
    
    # TODO: the game can end here if he got two aces and took 11 as both values (stupid case, but could happen)
    while True:
        # Ask the Player if they want to Hit or Stand
        player1_hits = Utils.wants_to_hit()

        if player1_hits:
            # He can draw an ACE here, consider that case
            player1.draw(my_deck)
            
            Utils.print_state(my_deck, dealer, player1, True)
            player1.define_aces_values()
            Utils.print_state(my_deck, dealer, player1, True)
            
            # Analyze if the player is busted
            if (player1.busted()):
                # The end of the game
                print("You've lost your bet. Wanna play again?")
                break
                # TODO: start game again
            else:
                continue               
        else:
            # Player wants to stand, let's break out of the loop
            break;
    
    # We can get to this point with the playe1 standing or losing


# Testing code
play()


#Ask the Player if they wish to Hit, and take another card
#If the Player's hand doesn't Bust (go over 21), ask if they'd like to Hit again.
#If a Player Stands, play the Dealer's hand. The dealer will always Hit until the Dealer's value meets or exceeds 17
#Determine the winner and adjust the Player's chips accordingly
#Ask the Player if they'd like to play again




    



    


Dealer
--------
Cards in deck: 48
Savings: $5000
Nine of Diamonds (9)|X|

Player1
--------
Savings: $400
Bet: $100
Ten of Hearts (10)|Nine of Spades (9)|
Would you like to hit[h] or stand[s]?s
